## **Random Forest Model Implementation for artificial data**


In [ ]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.value = value
        self.left = left
        self.right = right
        self.threshold = threshold
        self.feature = feature

    def check_node_leaf(self):
        return self.value is not None


**Defining the Decision Tree Model Class**

In [ ]:
class DecisionTree:
    # class initialization
    def __init__(self, minimum_samples_for_split=5, depth_maximum=5, number_of_features=None):
        self.minimum_samples_for_split = minimum_samples_for_split
        self.depth_maximum = depth_maximum
        self.number_of_features = number_of_features
        self.root = None

    # model fit method
    def fitting(self, X, y):
        if not self.number_of_features:
            self.number_of_features = X.shape[1]
        else:
            self.number_of_features = min(self.number_of_features, X.shape[1])

        self.root = self.build_decision_tree(X, y)

    # building of the decision tree
    def build_decision_tree(self, X, y, depth=0):
        number_of_samples, num_of_feats_procured = X.shape
        number_of_labels = len(np.unique(y))
        
        # 1. check the stopping criteria
        #Qn -> the order of the features selected by the tree and why it is correct

        # if either of these below is true, it is a stopping criteria
        if (number_of_labels == 1 or  number_of_samples < self.minimum_samples_for_split or depth >= self.depth_maximum):
            leaf_value = self.prominent_feature(y)
            tempRes = Node(value=leaf_value)
            return tempRes

        # to select random number of features in a single time
        feature_indexes = np.random.choice(num_of_feats_procured, self.number_of_features, replace=False)

        # 2. find the best split (to keep the tree growing)
        feat_best, thresh_best = self.calculate_split_best(X, y, feature_indexes)

        # create child nodes
        indexes_left_side, indexes_right_side = self.find_split(X[:, feat_best], thresh_best)
        subtree_right = self.build_decision_tree(X[indexes_right_side, :], y[indexes_right_side], depth+1)
        subtree_left = self.build_decision_tree(X[indexes_left_side, :], y[indexes_left_side], depth+1)

        return Node(feat_best, thresh_best, subtree_left, subtree_right)
    
    # model prediction code
    def prediction_calculation(self, X):
        return np.array([self.tree_traversal(x, self.root) for x in X])

    def tree_traversal(self, x, node):
        if node.check_node_leaf():
            return node.value

        if node.threshold >= x[node.feature]:
            return self.tree_traversal(x, node.left)

        return self.tree_traversal(x, node.right)

    # below is the list of all the helper functions

    # we find the best split available amongst all the available splits
    def calculate_split_best(self, X, y, feature_indexes):
        index_threshold, threshold_split = None, None
        gain_initial = -1
        

        for feat_idx in feature_indexes:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain for deciding the best-split
                gain = self.info_gain(y, X_column, thr)

                if gain > gain_initial:
                    gain_initial = gain
                    index_threshold = feat_idx
                    threshold_split = thr

        return index_threshold, threshold_split

    # computing the information-gain of the data
    def info_gain(self, y, X_column, threshold):
        parent_entropy = self.compute_entropy(y)

        # create children
        indexes_left_side, indexes_right_side = self.find_split(X_column, threshold)

        # if info gain is 0
        if len(indexes_left_side) == 0 or len(indexes_right_side) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        total_number = len(y)

        # getting the length/count of left and right indexes
        length_right = len(indexes_right_side)
        length_left = len(indexes_left_side)

        # num of left and right idxs entropies
        entropy_left, right_entropy = self.compute_entropy(y[indexes_left_side]), self.compute_entropy(y[indexes_right_side])

        #calc. child entropy
        child_entropy = (length_left/total_number) * entropy_left + (length_right/total_number) * right_entropy

        # calculate the Information Gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    # deciding the split of the data
    def find_split(self, X_column, split_thresh):
        # here the right and left indices split is decided
        indexes_right_side = np.argwhere(split_thresh < X_column).flatten()
        indexes_left_side = np.argwhere(split_thresh >= X_column).flatten()
        
        return indexes_left_side, indexes_right_side

    # calculation of entropy of data
    # minor change here with respect to real-world data as data processing(type casting) has to be done
    def compute_entropy(self, y):
        flatArray = [int(val) for sublist in y for val in sublist]
        
        counts = {}
        for val in flatArray:
            if val in counts:
                counts[val] += 1
            else:
                counts[val] = 1
        count = [counts.get(i, 0) for i in range(max(flatArray)+1)]

        ps = []
        for i in count:
            ps.append(round(i/len(count), 5))

        # entropy formula
        res = -np.sum([p * np.log(p) for p in ps if p > 0])
        return res

    # deciding the most common feature for deciding the leaf node
    def prominent_feature(self, y):
        ctr = {}

        # creating a dictionary to maintain frequency
        for i in y:
            if str(i) not in ctr:
                ctr[str(i)] = 1
            else:
                ctr[str(i)] += 1

        # class decision logic
        if '1' not in ctr:
            return 2
        elif '2' not in ctr:
            return 1
        elif '1' in ctr and '2' in ctr:
            if ctr['1'] > ctr['2']:
                return 1
            else:
                return 2


**Defining the RandomForest class**

In [ ]:
# from DecisionTrees import DecisionTree
import numpy as np
from collections import Counter

class RandomForest:
    # initializing the random forest
    def __init__(self, number_of_trees=10, depth_maximum=10, minimum_samples_for_split=2, n_feature=None):
        self.depth_maximum = depth_maximum
        self.minimum_samples_for_split = minimum_samples_for_split
        self.number_of_features = n_feature
        self.number_of_trees = number_of_trees
        self.trees = []

    # model fit
    def fitting(self, X, y):
        self.trees = []
        for _ in range(self.number_of_trees):
            tree_result = DecisionTree(depth_maximum=self.depth_maximum, minimum_samples_for_split= self.minimum_samples_for_split, number_of_features= self.number_of_features)
            samples_X, samples_Y = self.sampling_bootstrap(X, y)
            tree_result.fitting(samples_X, samples_Y)
            self.trees.append(tree_result)

    # performing bootstrap smpling
    def sampling_bootstrap(self, X, y):
        n_samples = 12
        # POINT FOR RANDOMEESS  IN DATA
        idxs = np.random.choice(X.shape[0], n_samples, replace=True)
        return X[idxs], y[idxs]

    # calculating the prominent feature
    def prominent_feature(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
        
    # calculating the prediction
    def prediction_calculation(self, X):
        predictions_results = np.array([tree.prediction_calculation(X) for tree in self.trees])
        prediction_of_trees = np.swapaxes(predictions_results, 0, 1)
        predictions_results = np.array([self.prominent_feature(pred) for pred in prediction_of_trees])
        return predictions_results

## The below code is for testing the above implementations

**Adding the necessary imports**

In [ ]:
import numpy as np
# from RandomForests import RandomForest
import pandas as pd

# for pre-processing, we are defining the MinMaxScaler
class MinMaxScaler:
    def __init__(self, feature_range=(0, 1)):
        self.feature_range = feature_range
    
    def fit(self, data):
        self.min_values = np.min(data, axis=0)
        self.max_values = np.max(data, axis=0)
        
    def transform(self, data):
        scaled_data = (data - self.min_values) / (self.max_values - self.min_values)
        scaled_data = scaled_data * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return scaled_data
    
    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)


**Necessary data imports**

In [ ]:
data1 = pd.read_csv('./implementation_correctness_dataset.csv')
data2 = pd.read_csv('./implementation_correctness_dataset-test.csv')

**Preparing the data**

In [ ]:
# train data
X1 = data1.iloc[:, [0,1]].values
y1 = data1.iloc[:, [2]].values

# test data
X_test2 = data2.iloc[:, [0,1]].values
y_test2 = data2.iloc[:, [2]].values

X1d = np.array(X1)
y1d = np.array(y1)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data_X1 = scaler.fit_transform(X1)
scaled_data_y1 = scaler.fit_transform(y1)


**Defining accuracy**

**Executing the Random forest model for the prediction**

In [ ]:
clf = RandomForest(number_of_trees=20)
clf.fitting(scaled_data_X1, scaled_data_y1)
predictions = clf.prediction_calculation(X_test2)
print('Class predictions on artificial dataset - Class:', predictions)

Class predictions on artificial dataset - Class: [2]
